In [26]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


In [27]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001


In [28]:
# MNIST Dataset
train_dataset = dsets.MNIST(root='../data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='../data/',
                           train=False, 
                           transform=transforms.ToTensor())


Files already downloaded


In [29]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [30]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [31]:
cnn = CNN()


In [32]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)


In [33]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))


Epoch [1/5], Iter [100/600] Loss: 0.2739
Epoch [1/5], Iter [200/600] Loss: 0.1497
Epoch [1/5], Iter [300/600] Loss: 0.0468
Epoch [1/5], Iter [400/600] Loss: 0.0679
Epoch [1/5], Iter [500/600] Loss: 0.0619
Epoch [1/5], Iter [600/600] Loss: 0.0707
Epoch [2/5], Iter [100/600] Loss: 0.0603
Epoch [2/5], Iter [200/600] Loss: 0.0632
Epoch [2/5], Iter [300/600] Loss: 0.0086
Epoch [2/5], Iter [400/600] Loss: 0.0283
Epoch [2/5], Iter [500/600] Loss: 0.1828
Epoch [2/5], Iter [600/600] Loss: 0.0286
Epoch [3/5], Iter [100/600] Loss: 0.0642
Epoch [3/5], Iter [200/600] Loss: 0.0358
Epoch [3/5], Iter [300/600] Loss: 0.0321
Epoch [3/5], Iter [400/600] Loss: 0.0054
Epoch [3/5], Iter [500/600] Loss: 0.0393
Epoch [3/5], Iter [600/600] Loss: 0.0314
Epoch [4/5], Iter [100/600] Loss: 0.0313
Epoch [4/5], Iter [200/600] Loss: 0.0452
Epoch [4/5], Iter [300/600] Loss: 0.0291
Epoch [4/5], Iter [400/600] Loss: 0.0153
Epoch [4/5], Iter [500/600] Loss: 0.0572
Epoch [4/5], Iter [600/600] Loss: 0.0362
Epoch [5/5], Ite

In [34]:
# Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))


Test Accuracy of the model on the 10000 test images: 99 %
